In [23]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [25]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/turfdiddy/Desktop/Bootcamp_ds:ml/Week_7/Day_1/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [26]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [27]:
# Your code
X = data['text']
y = data['label'] 

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,            # features and labels
    test_size=0.2,   # 20% for testing (adjust as needed)
    random_state=42, # ensures reproducibility
    stratify=y       # optional: keeps class distribution balanced
)

## Data Preprocessing

In [30]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [31]:
# Your code
X_train


442    Dear=2C Good day hope fine=2Cdear am writting ...
962    FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...
971                                             Will do.
190    FROM THE DESK OF DR.ADAMU  ISMALERAUDITING AND...
551    Dear Friend, My name is LOI C.ESTRADA,The wife...
                             ...                        
625    <P><A href=3D"http://www.rte.ieews/2000/0418/z...
53     IV><DIV>Dear,<BR>&nbsp;</DIV><DIV>It is my ple...
728    Will do. Think we are in good shape. Immelt In...
664    With your permission we are changing the state...
798                      Yes we will. Thank you Courtney
Name: text, Length: 800, dtype: object

In [32]:
from bs4 import BeautifulSoup
import re

def clean_html_with_bs4(html_text):
    # Parse the HTML
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Remove script and style elements completely
    for script in soup(["script", "style"]):
        script.decompose()
    
    # Get text and clean up whitespace
    text = soup.get_text()
    
    # Clean up extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [33]:
data["text"]=data["text"].apply(clean_html_with_bs4)

/var/folders/24/ngr8p82d5634jzq4nx8yv96c0000gn/T/ipykernel_64547/3775883403.py:6: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html_text, 'html.parser')


In [39]:
data

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [49]:
import re

def preprocess_text(text):
    # Convert to lowercase first (makes other operations easier)
    text = text.lower()
    
    # Remove prefixed 'b' (handles cases like b'text' from byte strings)
    text = re.sub(r'^b\'|\'$', '', text)  # Removes b' from start and ' from end
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove special characters (keep only letters and spaces)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Remove single characters (but not at word boundaries to avoid issues)
    text = re.sub(r'\b[a-z]\b', '', text)
    
    # Remove single characters from the start of the text
    text = re.sub(r'^[a-z]\s*', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing spaces
    text = text.strip()
    
    return text

In [40]:
# Your code
data["text"] = data["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", " ", str(x)))

In [41]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\d+", " ", x))

In [42]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\b[a-zA-Z]\b", " ", x))

In [43]:
data["text"] = data["text"].apply(lambda x: re.sub(r"^[a-zA-Z]\s+", "", x))

In [44]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\s+", " ", x).strip())

In [45]:
data["text"] = data["text"].apply(lambda x: x.lower())

In [46]:
data["text"] = data["text"].apply(lambda x: re.sub(r"\bb\w*", "", x))

## Now let's work on removing stopwords
Remove the stopwords.

In [48]:
# Your code
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    
    words = word_tokenize(text)
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    return " ".join(filtered_words)

X_train_final = X_train.apply(remove_stopwords)
print(X_train_final.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/turfdiddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/turfdiddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


442    Dear=2C Good day hope fine=2Cdear writting mai...
962    MR HENRY KABORETHE CHIEF AUDITOR INCHARGEFOREI...
971                                                    .
190    DESK DR.ADAMU ISMALERAUDITING ACCOUNTING MANAG...
551    Dear Friend , name LOI C.ESTRADA , wife Mr. JO...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [50]:
# Your code
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download required data
nltk.download('wordnet')
nltk.download('omw-1.4')  # For better lemmatization

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Tokenize the text
    words = word_tokenize(text)
    
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join back to text
    return ' '.join(lemmatized_words)

# Apply to your data
X_train_lemmatized = X_train_final.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/turfdiddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/turfdiddy/nltk_data...


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create the vectorizer
vectorizer = CountVectorizer(max_features=1000)  # Top 1000 most frequent words

# Fit and transform your text data
X_bow = vectorizer.fit_transform(X_train_lemmatized)

# Get feature names (the vocabulary)
feature_names = vectorizer.get_feature_names_out()

# Convert to dense array if you want to see it as a DataFrame
X_bow_df = pd.DataFrame(X_bow.toarray(), columns=feature_names)

print(f"Shape of BoW matrix: {X_bow.shape}")
print(f"Vocabulary size: {len(feature_names)}")
print("\nFirst few features:")
print(feature_names[:10])

Shape of BoW matrix: (800, 1000)
Vocabulary size: 1000

First few features:
['00' '000' '09' '0d' '10' '100' '11' '12' '13' '14']


In [55]:
# Check what we're working with
print("X_train_lemmatized type:", type(X_train_lemmatized))
print("y_train type:", type(y_train))
print("y_train unique values:", y_train.unique())
print("Shape of data:", X_train_lemmatized.shape, y_train.shape)

X_train_lemmatized type: <class 'pandas.core.series.Series'>
y_train type: <class 'pandas.core.series.Series'>
y_train unique values: [1 0]
Shape of data: (800,) (800,)


In [58]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create BoW matrix
vectorizer = CountVectorizer(max_features=1000)
X_bow = vectorizer.fit_transform(X_train_lemmatized)
feature_names = vectorizer.get_feature_names_out()

# Convert to dense array to avoid sparse matrix issues
X_bow_dense = X_bow.toarray()

# Create boolean masks - convert Series to numpy arrays
spam_mask = (y_train == 1).values  # Convert pandas Series to numpy array
ham_mask = (y_train == 0).values

# Get word frequencies for each class
spam_word_freq = X_bow_dense[spam_mask].sum(axis=0)
ham_word_freq = X_bow_dense[ham_mask].sum(axis=0)

# Create DataFrames with results
spam_words_df = pd.DataFrame({
    'word': feature_names, 
    'frequency': spam_word_freq
}).sort_values('frequency', ascending=False)

ham_words_df = pd.DataFrame({
    'word': feature_names, 
    'frequency': ham_word_freq
}).sort_values('frequency', ascending=False)

# Display results
print("TOP 10 WORDS IN SPAM EMAILS (label = 1):")
print(spam_words_df.head(10))
print(f"\nNumber of ham emails: |{ham_mask.sum()}")

TOP 10 WORDS IN SPAM EMAILS (label = 1):
            word  frequency
39            2e       1595
188           br        875
602        money        755
36            2c        722
86       account        658
164         bank        611
411         fund        560
915  transaction        428
192     business        409
620         nbsp        387

Number of ham emails: |446


## Extra features

In [63]:
# If your 'text' column already has all the preprocessing done:
data['preprocessed_text'] = data['text']  # Just rename/copy it

In [64]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data['money_mark'] = data['preprocessed_text'].str.contains(money_simbol_list)*1
data['suspicious_words'] = data['preprocessed_text'].str.contains(suspicious_words)*1
data['text_len'] = data['preprocessed_text'].apply(lambda x: len(x)) 

data['money_mark'] = data['preprocessed_text'].str.contains(money_simbol_list)*1
data['suspicious_words'] = data['preprocessed_text'].str.contains(suspicious_words)*1
data['text_len'] = data['preprocessed_text'].apply(lambda x: len(x)) 

data.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
0,dear sir strictly private proposal am mike ch...,1,dear sir strictly private proposal am mike ch...,1,1,2089
1,will do,0,will do,0,0,7
2,nora cheryl has emailed dozens of memos about ...,0,nora cheryl has emailed dozens of memos about ...,0,0,193
3,dear sir fmadam know that this proposal might ...,1,dear sir fmadam know that this proposal might ...,1,1,1941
4,fyi,0,fyi,0,0,3


In [62]:
# Check your column names
print("Column names in your dataframe:")
print(data.columns.tolist())
print("\nFirst few rows:")
print(data.head())

Column names in your dataframe:
['text', 'label']

First few rows:
                                                text  label
0  dear sir strictly private  proposal am mike ch...      1
1                                            will do      0
2  nora cheryl has emailed dozens of memos about ...      0
3  dear sir fmadam know that this proposal might ...      1
4                                                fyi      0


In [60]:
# Check what dataframes you have in memory
print("Available variables:")
print([var for var in dir() if not var.startswith('_') and 'DataFrame' in str(type(globals().get(var, '')))])

Available variables:
['X_bow_df', 'data', 'ham_words_df', 'spam_words_df']


## How would work the Bag of Words with Count Vectorizer concept?

In [65]:
# Your code
vectorizer = CountVectorizer(max_features=1000)
vectorizer.fit(X_train_lemmatized)  # Learns vocabulary from training data

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [66]:
X_bow = vectorizer.transform(X_train_lemmatized)  # Converts text to numbers

In [67]:
X_bow = vectorizer.fit_transform(X_train_lemmatized)  # Learn + convert in one step

In [68]:
# Example with 3 documents:
docs = [
    "free money transfer",
    "meeting tomorrow office", 
    "free account transfer money"
]

# Step 1: Build vocabulary
vocabulary = {"free": 0, "money": 1, "transfer": 2, "meeting": 3, "tomorrow": 4, "office": 5, "account": 6}

# Step 2: Count words in each document
# Doc 1: [1, 1, 1, 0, 0, 0, 0]  # free=1, money=1, transfer=1, others=0
# Doc 2: [0, 0, 0, 1, 1, 1, 0]  # meeting=1, tomorrow=1, office=1, others=0  
# Doc 3: [1, 1, 1, 0, 0, 0, 1]  # free=1, money=1, transfer=1, account=1, others=0

In [69]:
CountVectorizer(
    max_features=1000,      # Keep only top 1000 most frequent words
    min_df=2,              # Ignore words appearing in < 2 documents
    max_df=0.95,           # Ignore words appearing in > 95% of documents
    ngram_range=(1,1)      # Use single words (1,2) would include bigrams
)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [70]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Load/create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Keep top 1000 features
    min_df=2,          # Ignore words in < 2 documents  
    max_df=0.95        # Ignore words in > 95% of documents
)

# Vectorize your dataset (fit and transform)
X_tfidf = tfidf_vectorizer.fit_transform(X_train_lemmatized)

# Print the shape
print(f"Shape of TF-IDF vectorized dataset: {X_tfidf.shape}")
print(f"Number of documents: {X_tfidf.shape[0]}")
print(f"Number of features (vocabulary size): {X_tfidf.shape[1]}")

# Optional: See some of the features
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"First 10 features: {feature_names[:10]}")

Shape of TF-IDF vectorized dataset: (800, 1000)
Number of documents: 800
Number of features (vocabulary size): 1000
First 10 features: ['00' '000' '09' '0d' '10' '100' '11' '12' '13' '14']


## And the Train a Classifier?

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


In [78]:
# Split your data (if not already split)
X_train_tfidf, X_test_tfidf, y_train_split, y_test_split = train_test_split(
    X_tfidf, y_train, test_size=0.2, random_state=42
)

In [79]:
from sklearn.ensemble import RandomForestClassifier

# Use Random Forest instead of XGBoost
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

# Train the classifier
print("Training Random Forest classifier...")
rf_classifier.fit(X_train_tfidf, y_train_split)

# Make predictions
y_pred = rf_classifier.predict(X_test_tfidf)

# Evaluate performance
accuracy = accuracy_score(y_test_split, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_split, y_pred, target_names=['Ham', 'Spam']))

Training Random Forest classifier...

Accuracy: 0.9187

Classification Report:
              precision    recall  f1-score   support

         Ham       0.90      0.97      0.93        89
        Spam       0.95      0.86      0.90        71

    accuracy                           0.92       160
   macro avg       0.92      0.91      0.92       160
weighted avg       0.92      0.92      0.92       160



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code